# Ein Hybridmodell mit LSTM-AE, DBSCAN und IF

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.metrics import classification_report
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from matplotlib import pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
import logging
from tensorflow.keras.layers import TimeDistributed, Dense, LSTM, Dropout, LayerNormalization, Input, Add, Bidirectional, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_curve
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import balanced_accuracy_score
import sys
from keras.layers import LSTM, Dropout, LayerNormalization, Add, TimeDistributed, Dense
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, average_precision_score
from tensorflow.keras import backend as K

## 1. Vorbereitung

In [ ]:
# import torch
# print(torch.cuda.is_available())
# print(torch.cuda.get_device_name(0))

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import os
import tensorflow as tf

num_threads = os.cpu_count()
tf.config.threading.set_inter_op_parallelism_threads(num_threads)
tf.config.threading.set_intra_op_parallelism_threads(num_threads)
print(f"Verwendete CPU Threads: {num_threads}")

In [ ]:
from sklearn.model_selection import StratifiedKFold

n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [ ]:
# from dotenv import load_dotenv
# import requests
# import os

# event_url = os.environ.get("EVENT_URL")

# def get_all_events(token):
#     headers = {'Authorization': f'Bearer {token}'}
#     all_events = []
#     params = {'max': 5000}
#     resp = requests.get(event_url, headers=headers, params=params)
#     resp.raise_for_status()
#     batch = resp.json()
#     all_events.extend(batch)

#     return all_events

# server_url = os.environ.get("URL")
# client_id = os.environ.get("CLIENT_ID")
# client_secret = os.environ.get("CLIENT_SECRET")
# token_url = os.environ.get("TOKEN_URL")
# keycloak_url = os.environ.get("keycloak_url")
# authRealm = os.environ.get("authRealm")
# realm = os.environ.get("realm")

# load_dotenv()

# def get_token():
#     data = {
#         'keycloak_url': keycloak_url,
#         'authRealm': authRealm,
#         'realm': realm,
#         'client_id': client_id,
#         'client_secret': client_secret,
#         'grant_type': 'client_credentials'
#         }
#     resp = requests.post(token_url, data=data)
#     resp.raise_for_status()
#     token = resp.json()['access_token']
#     return token

# token = get_token()
# events = get_all_events(token)

In [ ]:
# from datetime import datetime
# import json


# filename = datetime.now().strftime("keycloak_events_%Y-%m-%d.jsonl")
# with open(filename, "a") as f:
#     for event in events:
#         f.write(json.dumps(event) + "\n")

#### Daten laden

In [ ]:
data_train = pd.read_json("train_logs.jsonl", lines=True)
data_test = pd.read_json("test_logs.jsonl", lines=True)
data_val = pd.read_json("val_logs.jsonl", lines=True)

In [ ]:
X_train_full = pd.DataFrame(data_train)
X_test_full = pd.DataFrame(data_test)
X_val_full = pd.DataFrame(data_val)

# data_test['label'] = data_test['details'].apply(lambda x: x.get('label', None))
# data_val['label'] = data_val['details'].apply(lambda x: x.get('label', None))
y_test_full = data_test["label"]
y_val_full = data_val["label"]

#### Automatisch alles in Numerische oder Kategorische Daten einteilen

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

def is_missing(val):
    return val is None or (isinstance(val, float) and np.isnan(val))

def flatten_log_entry(log, parent_key='', sep='.'):
    items = []
    for k, v in log.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_log_entry(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def auto_encode_features(logs, one_hot_numeric=False, label_encoders=None, onehot_encoders=None, fit=True):
    # Logs flach machen
    flat_logs = [flatten_log_entry(log) for log in logs]

    if label_encoders is None:
        label_encoders = {}
    if onehot_encoders is None:
        onehot_encoders = {}

    # Encoder vorbereiten
    for key in flat_logs[0].keys():
        values = []
        for log in flat_logs:
            val = log.get(key)

            if is_missing(val):
                val = "__MISSING__"
            elif isinstance(val, (list, dict)):
                print(f"Feature '{key}' enthält nicht-skalaren Wert – wird als String gespeichert: {val}")
                val = str(val)

            values.append(val)

        if fit:
            try:
                le = LabelEncoder()
                le.fit(values)
                label_encoders[key] = le
            except Exception as e:
                print(f"Fehler beim Enkodieren von Feature '{key}': {e}")
                continue

    # Feature-Werte transformieren
    encoded_logs = []
    for log in flat_logs:
        encoded_log = {}
        for key, le in label_encoders.items():
            val = log.get(key, "__MISSING__")
            if is_missing(val):
                val = "__MISSING__"
            elif isinstance(val, (list, dict)):
                val = str(val)
            try:
                encoded_log[key] = le.transform([val])[0]
            except ValueError:
                encoded_log[key] = -1  # unbekannte Kategorie
        encoded_logs.append(encoded_log)

    return encoded_logs, label_encoders, onehot_encoders

#### Werte aus vorheriger Funktion vereinigen

In [ ]:
def dicts_to_feature_matrix(encoded_logs):
    feature_names = sorted({key for d in encoded_logs for key in d.keys()})

    X = np.zeros((len(encoded_logs), len(feature_names)), dtype=np.float32)

    for i, d in enumerate(encoded_logs):
        for j, feat in enumerate(feature_names):
            if feat in d:
                X[i, j] = d[feat]

    return X, feature_names

In [ ]:
logs_train = X_train_full.to_dict(orient='records')
#logs_train = events
logs_test = X_test_full.to_dict(orient='records')
logs_val = X_val_full.to_dict(orient='records')

encoded_logs, label_encoders, onehot_encoders = auto_encode_features(
    logs_train, one_hot_numeric=False, fit=True
)
X_train, feature_names = dicts_to_feature_matrix(encoded_logs)

encoded_test_logs, _, _ = auto_encode_features(
    logs_test, one_hot_numeric=False,
    label_encoders=label_encoders, onehot_encoders=onehot_encoders, fit=False
)
X_test, _ = dicts_to_feature_matrix(encoded_test_logs)

encoded_val_logs, _, _ = auto_encode_features(
    logs_val, one_hot_numeric=False,
    label_encoders=label_encoders, onehot_encoders=onehot_encoders, fit=False
)
X_val, _ = dicts_to_feature_matrix(encoded_val_logs)

#### Skalierung

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)

### Parameter

In [ ]:
seq_length = 25
batch_size = 32

encoder_layers = [128, 64]
decoder_layers = [64, 128]

dropout_rate = 0.1
learning_rate = 0.001

### Sequenzen

In [ ]:
class SequenceToSequenceGenerator(tf.keras.utils.Sequence):
    def __init__(self, data, seq_length, batch_size, pad_last=True):
        self.data = data
        self.seq_length = seq_length
        self.batch_size = batch_size
        self.pad_last = pad_last

        self.indices = np.arange(len(data) - seq_length)
        
        # Wenn Padding aktiv ist und etwas übrig bleibt
        if pad_last and (len(data) % seq_length != 0):
            self.include_last = True
        else:
            self.include_last = False

    def __len__(self):
        base = (len(self.indices) + self.batch_size - 1) // self.batch_size
        return base + (1 if self.include_last else 0)

    def __getitem__(self, idx):
        if idx < len(self) - 1 or not self.include_last:
            batch_idx = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
            X_batch = np.array([self.data[i:i + self.seq_length] for i in batch_idx])
        else:
            # letzte Sequenz mit Padding
            last_seq = self.data[-self.seq_length:]
            if len(last_seq) < self.seq_length:
                padding_len = self.seq_length - len(last_seq)
                padding = np.zeros((padding_len, self.data.shape[1]))
                last_seq = np.vstack((last_seq, padding))
            X_batch = np.expand_dims(last_seq, axis=0)

        return X_batch, X_batch

train_gen = SequenceToSequenceGenerator(X_train_scaled, seq_length, batch_size)
val_gen = SequenceToSequenceGenerator(X_val_scaled, seq_length, batch_size)
test_gen = SequenceToSequenceGenerator(X_test_scaled, seq_length, batch_size)

## 2. LSTM-AE implementieren

In [ ]:
from tensorflow.keras.layers import Input, Bidirectional, LSTM, RepeatVector, TimeDistributed, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

n_features = X_train_scaled.shape[1]

inputs = Input(shape=(seq_length, n_features))
x = inputs

# Tieferer Encoder mit bidirektionalen LSTMs
for units in encoder_layers:
    x = Bidirectional(LSTM(units, return_sequences=True))(x)
    x = Dropout(dropout_rate)(x)

# Letzte Encoder-Schicht ohne return_sequences, nur den letzten Output
encoded = Bidirectional(LSTM(encoder_layers[-1]))(x)

# Tieferer Decoder
x = RepeatVector(seq_length)(encoded)
for units in decoder_layers:
    x = Bidirectional(LSTM(units, return_sequences=True))(x)
    x = Dropout(dropout_rate)(x)

outputs = TimeDistributed(Dense(n_features, activation='linear'))(x)

model = Model(inputs, outputs)
model.compile(optimizer=Adam(learning_rate), loss='mse')
model.summary()

In [ ]:
#early_stop = EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
model.fit(train_gen, epochs=50, validation_data=val_gen)

### Umwandlung der Daten für nächste Modelle: Rekonstruktionsfehler berechnen

In [ ]:
def get_reconstruction_errors(gen, model):
    errors = []
    for i in range(len(gen)):
        X_batch, _ = gen[i]
        pred = model.predict(X_batch, verbose=0)
        batch_errors = np.mean(np.square(X_batch - pred), axis=(1, 2))
        errors.extend(batch_errors)
    return np.array(errors)

train_errors = get_reconstruction_errors(train_gen, model)
val_errors = get_reconstruction_errors(val_gen, model)
test_errors = get_reconstruction_errors(test_gen, model)
#reconstruction_errors = get_reconstruction_errors(test_gen, model)

In [ ]:
# from sklearn.metrics import precision_recall_curve, precision_recall_fscore_support, roc_auc_score
# import numpy as np

y_test_seq = np.array([
    int(np.any(y_test_full[i : i + seq_length] == 1))
    for i in range(len(test_errors))
])

#true_labels = y_test_seq.astype(int)

# precisions, recalls, thresholds = precision_recall_curve(true_labels, test_errors)
# f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-10)

# best_idx = np.argmax(f1_scores)
# optimal_threshold = thresholds[best_idx]

# y_pred = (test_errors > optimal_threshold).astype(int)

# precision, recall, f1, _ = precision_recall_fscore_support(true_labels, y_pred, average='binary')
# roc_auc = roc_auc_score(true_labels, test_errors)

# print(f"Best Threshold by F1: {optimal_threshold:.5f}")
# print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1: {f1:.3f}, ROC-AUC: {roc_auc:.3f}")

In [ ]:
# scaler_rec = StandardScaler()
# scaler_rec.fit(train_errors.reshape(-1, 1))
# test_errors_scaled = scaler_rec.transform(test_errors.reshape(-1, 1))

# Modell 1

## Isolation Forest

In [ ]:
from sklearn.metrics import accuracy_score

iforest = IsolationForest(random_state=42, contamination=0.0008, max_samples = 'auto')
iforest.fit(train_errors.reshape(-1, 1))

scores_if = iforest.decision_function(test_errors.reshape(-1, 1))
precisions_if, recalls_if, thresholds_if = precision_recall_curve(y_test_seq, -scores_if)
best_f1_scores_if = 2 * (precisions_if * recalls_if) / (precisions_if + recalls_if + 1e-10)
best_idx_if = np.argmax(best_f1_scores_if)
optimal_threshold_if = thresholds_if[best_idx_if]
y_pred_if = ((-scores_if) > optimal_threshold_if).astype(int)

precision_if, recall_if, f1_if, _ = precision_recall_fscore_support(y_test_seq, y_pred_if, average='binary')
roc_auc_if = roc_auc_score(y_test_seq, -scores_if)
pr_auc_if = average_precision_score(y_test_seq, y_pred_if)
fpr, tpr, _ = roc_curve(y_test_seq, y_pred_if)
accuracy_if = accuracy_score(y_test_seq, y_pred_if)

mcc_if = matthews_corrcoef(y_test_seq, y_pred_if)
balanced_acc_if = balanced_accuracy_score(y_test_seq, y_pred_if)

print(f"IsolationForest Best Threshold by F1: {optimal_threshold_if:.4f}")
print(f"Precision: {precision_if:.4f}, Recall: {recall_if:.4f}, F1: {f1_if:.4f}, ROC-AUC: {roc_auc_if:.4f}, AUC-PR: {pr_auc_if:.4f}")
print(f"Matthews Correlation Coefficient: {mcc_if:.4f}, Balanced Accuracy: {balanced_acc_if:.4f}, Accuracy: {accuracy_if:.4f}")

# Modell 2

## One-Class SVM

In [ ]:
ocsvm = OneClassSVM(nu = 0.0008, gamma='scale', kernel = 'rbf')
ocsvm.fit(train_errors.reshape(-1,1))

scores_ocsvm = ocsvm.decision_function(test_errors.reshape(-1, 1))  # kleiner = anomal
precisions_ocsvm, recalls_ocsvm, thresholds_ocsvm = precision_recall_curve(y_test_seq, -scores_ocsvm)
f1_scores_ocsvm = 2 * (precisions_ocsvm * recalls_ocsvm) / (precisions_ocsvm + recalls_ocsvm + 1e-10)
best_idx_ocsvm = np.argmax(f1_scores_ocsvm)
optimal_threshold_ocsvm = thresholds_ocsvm[best_idx_ocsvm]

# Vorhersagen auf Testset
y_pred_ocsvm = ((-scores_ocsvm) > optimal_threshold_ocsvm).astype(int)

# Metriken
precision_ocsvm, recall_ocsvm, f1_ocsvm, _ = precision_recall_fscore_support(y_test_seq, y_pred_ocsvm, average='binary')
roc_auc_ocsvm = roc_auc_score(y_test_seq, -scores_ocsvm)
fpr, tpr, _ = roc_curve(y_test_seq, y_pred_ocsvm)
pr_auc_ocsvm = average_precision_score(y_test_seq, y_pred_ocsvm)
accuracy_ocsvm = accuracy_score(y_test_seq, y_pred_ocsvm)

mcc_ocsvm = matthews_corrcoef(y_test_seq, y_pred_ocsvm)
balanced_acc_ocsvm = balanced_accuracy_score(y_test_seq, y_pred_ocsvm)

print(f"One-Class SVM Best Threshold by F1: {optimal_threshold_ocsvm:.4f}")
print(f"Precision: {precision_ocsvm:.4f}, Recall: {recall_ocsvm:.4f}, F1: {f1_ocsvm:.4f}, ROC-AUC: {roc_auc_ocsvm:.4f}, AUC-PR: {pr_auc_ocsvm:.4f},MCC: {mcc_ocsvm:.4f}, Balanced Accuracy: {balanced_acc_ocsvm:.4f}, Accuracy: {accuracy_ocsvm:.4f}")

# Modell 3

## DBSCAN

In [ ]:
for eps in np.linspace(0.01, 1.0, 100):
    dbscan = DBSCAN(min_samples=3, metric='euclidean', eps=eps)
    labels = dbscan.fit_predict(test_errors.reshape(-1, 1))
    n_anomalies = np.sum(labels == -1)
    if abs(n_anomalies - 31) <= 2:
        print(f'Passender eps-Wert: {eps}, Anomalien: {n_anomalies}')
        break

y_pred_dbscan = (labels == -1).astype(int)

# Metriken berechnen
precision_dbscan, recall_dbscan, f1_dbscan, _ = precision_recall_fscore_support(y_test_seq, y_pred_dbscan, average='binary')
roc_auc_dbscan = roc_auc_score(y_test_seq, y_pred_dbscan)
pr_auc_dbscan = average_precision_score(y_test_seq, y_pred_dbscan)
accuracy_dbscan = accuracy_score(y_test_seq, y_pred_dbscan)
mcc_dbscan = matthews_corrcoef(y_test_seq, y_pred_dbscan)
balanced_acc_dbscan = balanced_accuracy_score(y_test_seq, y_pred_dbscan)

print(f"Precision: {precision_dbscan:.4f}, Recall: {recall_dbscan:.4f}, F1: {f1_dbscan:.4f}")
print(f"MCC: {mcc_dbscan:.4f}, Balanced Accuracy: {balanced_acc_dbscan:.4f}")
print(f"ROC-AUC: {roc_auc_dbscan:.4f}, AUC-PR: {pr_auc_dbscan:.4f}, Accuracy: {accuracy_dbscan:.4f}")

In [ ]:
import pandas as pd
from statsmodels.multivariate.manova import MANOVA

# Daten für alle drei Algorithmen
data = {
    'algorithm': ['IsolationForest']*9 + ['OCSVM']*9 + ['DBSCAN']*9,
    'seq_len': [10]*3 + [25]*3 + [50]*3 + [10]*3 + [25]*3 + [50]*3 + [10]*3 + [25]*3 + [50]*3,
    'f1': [
        0.8534,0.8831,0.8583, 0.7512,0.7647,0.7651, 0.5944,0.5410,0.6009,
        0.8655,0.8831,0.8560, 0.7548,0.7651,0.7656, 0.6014,0.5901,0.6009,
        0.2585,0.2222,0.3590, 0.0846,0.0846,0.0920, 0.0506,0.0506,0.0506
    ],
    'mcc': [
        0.8553,0.8862,0.8574, 0.7574,0.7771,0.7747, 0.6069,0.5044,0.6030,
        0.8654,0.8862,0.8542, 0.7622,0.7747,0.7725, 0.5692,0.5570,0.6030,
        0.3817,0.3502,0.4637, 0.2059,0.2059,0.2150, 0.1553,0.1553,0.1553
    ],
    'balanced_acc': [
        0.8863,0.8984,0.9016, 0.8142,0.8130,0.8167, 0.7236,0.7568,0.7330,
        0.9017,0.8984,0.9053, 0.8144,0.8167,0.8205, 0.7966,0.7951,0.7330,
        0.5742,0.5625,0.6094, 0.5221,0.5221,0.5241, 0.5130,0.5130,0.5130
    ]
}

df = pd.DataFrame(data)

def manova_to_df(manova_results, algorithm_name):
    results = manova_results.mv_test().results
    rows = []
    for effect, val in results.items():
        if effect == 'Intercept':
            continue
        stats = val['stat']
        for test_name in ['Wilks\' lambda', 'Pillai\'s trace', 'Hotelling-Lawley trace', 'Roy\'s greatest root']:
            row = {
                'Algorithm': algorithm_name,
                'Effect': effect,
                'Test': test_name,
                'Value': round(stats.loc[test_name, 'Value'], 3),
                'F': round(stats.loc[test_name, 'F Value'], 3),
                'p': round(stats.loc[test_name, 'Pr > F'], 4)
            }
            rows.append(row)
    return pd.DataFrame(rows)


# Isolation Forest
manova_if = MANOVA.from_formula('f1 + mcc + balanced_acc ~ C(seq_len)', data=df[df.algorithm=='IsolationForest'])
df_if = manova_to_df(manova_if, 'IsolationForest')

# OCSVM
manova_ocsvm = MANOVA.from_formula('f1 + mcc + balanced_acc ~ C(seq_len)', data=df[df.algorithm=='OCSVM'])
df_ocsvm = manova_to_df(manova_ocsvm, 'OCSVM')

# DBSCAN
manova_dbscan = MANOVA.from_formula('f1 + mcc + balanced_acc ~ C(seq_len)', data=df[df.algorithm=='DBSCAN'])
df_dbscan = manova_to_df(manova_dbscan, 'DBSCAN')

# Alles zusammen
df_all = pd.concat([df_if, df_ocsvm, df_dbscan], ignore_index=True)
df_all

## Cross Validation Test

### Für LSTM-AE und IF

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import IsolationForest
import numpy as np

n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

all_precisions, all_recalls, all_f1s, all_roc_aucs = [], [], [], []

for fold, (train_idx, test_idx) in enumerate(kf.split(X_test_full)):
    print(f"Fold {fold + 1}/{n_splits}")

    model.fit(train_gen, validation_data=val_gen, epochs=50, callbacks=[early_stop], verbose=0)

    iforest = IsolationForest()
    iforest.fit(train_errors.reshape(-1, 1))

    test_preds_if = iforest.predict(test_errors.reshape(-1, 1))
    val_anomaly_if = (test_preds_if == -1).astype(int)

    y_test_seq_if = np.array([y_test_full[i + seq_length- 1] for i in range(len(val_anomaly_if))])
    true_labels_if = y_test_seq_if.astype(int)

    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, y_pred, average='binary')
    roc_auc = roc_auc_score(true_labels, iforest.decision_function(test_errors.reshape(-1, 1)) * -1)

    all_precisions.append(precision)
    all_recalls.append(recall)
    all_f1s.append(f1)
    all_roc_aucs.append(roc_auc)

    print(f"Mean Precision: {np.mean(all_precisions):.3f} ± {np.std(all_precisions):.3f}")
    print(f"Mean Recall: {np.mean(all_recalls):.3f} ± {np.std(all_recalls):.3f}")
    print(f"Mean F1-Score: {np.mean(all_f1s):.3f} ± {np.std(all_f1s):.3f}")
    print(f"Mean ROC-AUC: {np.mean(all_roc_aucs):.3f} ± {np.std(all_roc_aucs):.3f}")

### Für LSTM-AE und OCSVM

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import IsolationForest
import numpy as np

n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

all_precisions, all_recalls, all_f1s, all_roc_aucs = [], [], [], []

for fold, (train_idx, test_idx) in enumerate(kf.split(X_test_full)):
    print(f"Fold {fold + 1}/{n_splits}")

    model.fit(train_gen, validation_data=val_gen, epochs=50, callbacks=[early_stop], verbose=0)
    ocsvm = OneClassSVM(nu=0.005, gamma=50)
    ocsvm.fit(train_errors.reshape(-1, 1))

    test_preds_ocsvm = ocsvm.predict(test_errors.reshape(-1, 1))
    test_anomaly_ocsvm = (test_preds_ocsvm == -1).astype(int)

    y_test_seq_ocsvm = np.array([y_test_full[i + seq_length - 1] for i in range(len(test_anomaly_ocsvm))])
    true_labels_ocsvm = y_test_seq.astype(int)

    report_ocsvm = classification_report(true_labels_ocsvm, test_anomaly_ocsvm)

    all_precisions.append(precision)
    all_recalls.append(recall)
    all_f1s.append(f1)
    all_roc_aucs.append(roc_auc)

    print(f"Mean Precision: {np.mean(all_precisions):.3f} ± {np.std(all_precisions):.3f}")
    print(f"Mean Recall: {np.mean(all_recalls):.3f} ± {np.std(all_recalls):.3f}")
    print(f"Mean F1-Score: {np.mean(all_f1s):.3f} ± {np.std(all_f1s):.3f}")
    print(f"Mean ROC-AUC: {np.mean(all_roc_aucs):.3f} ± {np.std(all_roc_aucs):.3f}")

### Für LSTM-AE und DBSCAN

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import IsolationForest
import numpy as np

n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

all_precisions, all_recalls, all_f1s, all_roc_aucs = [], [], [], []

for fold, (train_idx, test_idx) in enumerate(kf.split(X_test_full)):
    print(f"Fold {fold + 1}/{n_splits}")

    model.fit(train_gen, validation_data=val_gen, epochs=50, callbacks=[early_stop], verbose=0)
    dbscan = DBSCAN(eps = 0.05, min_samples = 40)

    dbscan_labels = dbscan.fit_predict(test_errors.reshape(-1, 1))
    dbscan_anomaly = (dbscan_labels == -1).astype(int)

    y_test_seq_dbscan = np.array([y_test_full[i + seq_length - 1] for i in range(len(dbscan_anomaly))])
    true_labels_dbscan = y_test_seq_dbscan.astype(int)
    
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_f1s.append(f1)
    all_roc_aucs.append(roc_auc)

    print(f"Mean Precision: {np.mean(all_precisions):.3f} ± {np.std(all_precisions):.3f}")
    print(f"Mean Recall: {np.mean(all_recalls):.3f} ± {np.std(all_recalls):.3f}")
    print(f"Mean F1-Score: {np.mean(all_f1s):.3f} ± {np.std(all_f1s):.3f}")
    print(f"Mean ROC-AUC: {np.mean(all_roc_aucs):.3f} ± {np.std(all_roc_aucs):.3f}")